In [1]:
import requests
import base64
from urllib.parse import urlencode
import json
import datetime

#import constants from KEYS module. create a module called KEYS.py and Add your own Client ID and Client Secret to it
import KEYS

In [2]:
requests.__version__

'2.25.1'

In [3]:
#assign keys to values here

client_id = KEYS.CLIENT_ID
client_secret = KEYS.CLIENT_SECRET

In [4]:
#format and encode credential in to expected form for api

client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())
print(base64.b64decode(client_creds_b64).decode())
print(client_creds)
print(client_creds_b64.decode())

af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
af37f76b6c204b16b5f781ce1e54603d:cdc8546a2bdd42ea96f3fe6a860ac1d3
YWYzN2Y3NmI2YzIwNGIxNmI1Zjc4MWNlMWU1NDYwM2Q6Y2RjODU0NmEyYmRkNDJlYTk2ZjNmZTZhODYwYWMxZDM=


In [5]:
#Builds the parts of the querey
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = {
    "grant_type":"client_credentials",
}
token_headers = {
    "Authorization" : f"Basic {client_creds_b64.decode()}",
}

In [6]:
#makes GET request to spotify API
r = requests.post(token_url, headers=token_headers, data=token_data)

#validates result, then prints
valid_request = r.status_code in range(200,299)
print(r.json())

{'access_token': 'BQC5ESeH-TEQ5Xm5leOQKVDZ4bmGs9gW5eIxFWWQ2B2ElusaIh5OpbRDS1qdcqMn6tmAiAbwpnq8nxE6Ke4', 'token_type': 'Bearer', 'expires_in': 3600}


In [7]:
#uses validation boolean to decide if to parse
if valid_request:
    token_response_data = r.json() #convert json to dict
    now = datetime.datetime.now() # create datetime obj for now
    access_token = token_response_data['access_token'] #retrieve token from response
    expires_in = token_response_data['expires_in'] #retrieve token lifetime
    expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
    did_expire = expires < datetime.datetime.now() #boolean to rep if expired

In [8]:
did_expire

False

In [9]:
expires

datetime.datetime(2021, 6, 3, 9, 53, 11, 570684)

# Base API Class 

In [57]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentiatls(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentiatls()
        return  {
            "Authorization" : f"Basic {client_creds_b64}",
        }
    
    def get_token_data(self):
        return {
            "grant_type":"client_credentials",
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_headers = self.get_token_headers()
        token_data = self.get_token_data()
        r = requests.post(token_url, headers=token_headers, data=token_data)
        if r.status_code not in range(200,299):
            raise Exception("Could not Authenticate client")
            #return False
        data = r.json() #convert json to dict
        now = datetime.datetime.now() # create datetime obj for now
        access_token = data['access_token'] #retrieve token from response
        expires_in = data['expires_in'] #retrieve token lifetime
        print(expires_in)
        expires = now + datetime.timedelta(seconds=expires_in) #creates datetime for when token expires
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < datetime.datetime.now() #boolean to rep if expired
        return True

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers


    def get_resources(self, lookup_id, resource_type = 'albums', version = 'v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()


    def get_album(self, _id):
        return self.get_resources(_id, resource_type= 'albums')

    def get_artist(self, _id):
        return self.get_resources(_id, resource_type= 'artists')


    def base_search(self, query_params):
        headers = self.get_resource_header()
        
        endpoint = "https://api.spotify.com/v1/search"
        r = requests.get(endpoint, params=query_params, headers=headers)
         
        print(r.url)
        if r.status_code in range(200,299):
            return r.json()
        return {}

    def search(self, query=None, search_type = 'artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f'{k}:{v}' for k,v in query.items()])
        query_params = {
            "q":query,
            "type":search_type.lower()
        }
        
        return self.base_search(query_params)
    


In [58]:
client = SpotifyAPI(client_id, client_secret)

In [12]:
client.get_artist('28j8lBWDdDSHSSt5oPlsX2')

3600


{'external_urls': {'spotify': 'https://open.spotify.com/artist/28j8lBWDdDSHSSt5oPlsX2'},
 'followers': {'href': None, 'total': 691483},
 'genres': ['edm', 'electro house'],
 'href': 'https://api.spotify.com/v1/artists/28j8lBWDdDSHSSt5oPlsX2',
 'id': '28j8lBWDdDSHSSt5oPlsX2',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb5cc7aca0b30b4214a4dc154e',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051745cc7aca0b30b4214a4dc154e',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1785cc7aca0b30b4214a4dc154e',
   'width': 160}],
 'name': 'ZHU',
 'popularity': 76,
 'type': 'artist',
 'uri': 'spotify:artist:28j8lBWDdDSHSSt5oPlsX2'}

In [115]:
client.perform_auth()
client.access_token

3600


'BQASQBIyPlxvkODd0D_O2FP0BFFMjmsDDNsRw3I9NoHiM4Y03FF1ThbosQfqOyBWeQH4mCu3kvSe5gTvIzs'

In [69]:
client.search({ 'track': 'ONLY', 'artist': 'zhu'}, search_type= 'track')

https://api.spotify.com/v1/search?q=track%3AONLY+artist%3Azhu&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AONLY+artist%3Azhu&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/28j8lBWDdDSHSSt5oPlsX2'},
       'href': 'https://api.spotify.com/v1/artists/28j8lBWDdDSHSSt5oPlsX2',
       'id': '28j8lBWDdDSHSSt5oPlsX2',
       'name': 'ZHU',
       'type': 'artist',
       'uri': 'spotify:artist:28j8lBWDdDSHSSt5oPlsX2'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK'

In [121]:
client.get_album('50QKpfridaIORN2l2hfRFI')

  'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CA',
     'CH',
     'CI',
     'CL',
     'CM',
     'CO',
     'CR',
     'CV',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DJ',
     'DK',
     'DM',
     'DO',
     'DZ',
     'EC',
     'EE',
     'EG',
     'ES',
     'FI',
     'FJ',
     'FM',
     'FR',
     'GA',
     'GB',
     'GD',
     'GE',
     'GH',
     'GM',
     'GN',
     'GQ',
     'GR',
     'GT',
     'GW',
     'GY',
     'HK',
     'HN',
     'HR',
     'HT',
     'HU',
     'ID',
     'IE',
     'IL',
     'IN',
     'IS',
     'IT',
     'JM',
     'JO',
     'JP',
     'KE',
     'KG',
     'KH',
     'KI',
     'KM',
     'KN',
     'KR',
     'KW',
     'KZ',
     'LA',
     'LB',
     'LC',
     'LI',
     'LK',
     'LR',
     'LS',
     'LT',
     'LU',
     'LV',
     'MA',
     'MC',
     'MD',
     'ME',
     'MG',
     'MH',
     'MK',
     'ML',
     'MN',
     'MO',
     'MR',
     'MT',
     'MU',
     'MV',
     'MW',
  

In [ ]:
class():
    def __repr__(self):
        return f'some string representation of the obect {self.}